In [1]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import solve_ivp
from scipy.integrate import odeint

from HJK.dynamics.systems import simple2D, rk1, rk4
from HJK.basis import monomials
from deeptime.basis import Monomials
import PySINDY.pysindy as ps

c:\users\sriramk\downloads\hamilton_jacobi_koopman\hamilton-jacobi-koopman\deeptime\deeptime\util\platform.py:23: RuntimeWarning: There was a problem importing torch, treating it as unavailable. Stacktrace: [WinError 182] The operating system cannot run %1. Error loading "c:\Users\sriramk\.conda\envs\HJ_Koopman\lib\site-packages\torch\lib\shm.dll" or one of its dependencies.
  warnings.warn(f"There was a problem importing {modname}, treating it as unavailable. Stacktrace: {e}",


In [2]:
plt.rcParams['figure.figsize'] = [9, 6]
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [3]:
deeptime_lib = Monomials(p=1, d=2)
deeptime_lib.get_feature_names(["x1", "x2"])

['', 'x1', 'x2']

In [4]:
sindy_lib = ps.PolynomialLibrary()

In [5]:
from sklearn.preprocessing import PolynomialFeatures
library = PolynomialFeatures(degree=3)

In [15]:
d3s_monomials = monomials.monomials(2)
d3s_monomials.display(,2)

1.000001
